In [ ]:
import pandas as pd
import nltk
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

input_csv = "in/tables/filtered_lyrics.csv"
output_csv = "out/tables/topics_songs.csv"
output_csv_topics = "out/tables/topics_words.csv"

df = pd.read_csv(input_csv, encoding='utf-8')
vectorizer = TfidfVectorizer(min_df = 0.1)
tfidf = vectorizer.fit_transform(df['filtered_lyrics'])

nmf = NMF(n_components = 6)
topic_values = nmf.fit_transform(tfidf)

topics = []
# Iterate over each topic
for topic_num, topic in enumerate(nmf.components_):
    # Generate topic name
    topic_name = "Topic #{}".format(topic_num + 1)
    # Extract top 10 terms for the topic and join them into a string
    topic_words = " ".join([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])
    # Add topic as a dictionary to the list
    topics.append({'Topic': topic_name, 'Words': topic_words})

# Create DataFrame for topics
df_topics_to_csv = pd.DataFrame(topics)

# Save DataFrame to CSV file
df_topics_to_csv.to_csv(output_csv_topics, index=False, encoding='utf-8')

topic_labels = ["Topic #1","Topic #2","Topic #3","Topic #4","Topic #5","Topic #6"]
df_topics = pd.DataFrame(topic_values, columns = topic_labels)
songs = df.join(df_topics)
df = pd.DataFrame(songs)

# define tne main topik in song
df["topic"] = df[["Topic #1","Topic #2","Topic #3","Topic #4","Topic #5","Topic #6"]].idxmax(axis=1)
df = df[['song', 'artist', 'topic']]
df.to_csv(output_csv,index=False,encoding='utf-8')